In [35]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import Perceptron
import copy
import math
import cvxpy as cp

In [36]:
class Gentile():
    def __init__(self, p, alpha, B, C):
        self.alpha = alpha
        self.B = B
        self.C = C
        self.p = p
        self.q = self.p/(self.p - 1)
        
    def solve(self, X, y):
        dim = X.shape[1]
        self.initialize(dim)
        for _ in range(1000):
            self.forward(X, y)
            accuracy_metric = np.sum(y * (X @ self.weights) > 0)/X.shape[0]
            print("Accuracy: {}%".format(accuracy_metric))
        return self.weights
    
    def initialize(self, dim):
        self.weights = np.zeros(dim)
        self.k = 1
        
    def forward(self, X, y):
        self.gamma = self.B * np.sqrt(self.p - 1) * 1/np.sqrt(self.k)
    
        
        for t in range(X.shape[0]):
            if y[t] * np.dot(self.weights,X[t]) <= (1 - self.alpha) * self.gamma:
                self.eta = self.C/np.sqrt(self.p - 1) * 1/np.sqrt(self.k)
                w_prime = self.finv(self.f(self.weights)+ self.eta * y[t] * X[t])
                q_norm = np.linalg.norm(w_prime, ord=self.q)
                self.weights = w_prime/max(1, q_norm)
                self.k += 1
            
                
    def f(self, w):
        numerator = np.sign(w) * np.power(abs(w), self.q - 1)
        denominator = np.power(np.linalg.norm(w, ord=self.q), self.q - 2)
        return numerator/denominator
    
    def finv(self, theta):
        numerator = np.sign(theta) * np.power(abs(theta), self.p - 1)
        denominator = np.power(np.linalg.norm(theta, ord=self.p), self.p - 2)
        return numerator/denominator
        
    

In [48]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

class Pnorm_accelerated():
    def __init__(self, p):

        self.p = p
        self.q = p/(p-1)
        self.eta_w = 1/(2 * self.q - 2)
    
    def initialize(self, X, y):
        self.ws = np.array([])
        self.probs = np.array([])
        self.probs = np.append(self.probs, np.asarray([1] * X.shape[0])/X.shape[0])
        self.probs = self.probs.reshape(1, X.shape[0])
        
    
    def iterate(self, X, y):
        self.initialize(X, y)
        M = X * y[:, np.newaxis]
        percent_accuracy = 0
        for t in tqdm(range(1000), desc="Acc: {0:.3g}".format(percent_accuracy)):
            w = cp.Variable(len(X[0]))
            banana = -1 * np.sum(self.probs @ M, axis=0) @ w + -1 * self.probs[-1] @ M @ w + self.eta_w/2 * cp.square(cp.atoms.pnorm(w))
            problem = cp.Problem(cp.Minimize(banana) )
            try:
                problem.solve()
            except:
                problem.solve(solver="SCS")
            self.w = w.value
            
            if len(self.ws) == 0:
                self.ws = np.asarray(self.w).reshape(1, X.shape[0])
            else:
                self.ws = np.vstack([self.ws, self.w.reshape(1, X.shape[0])])
            
            prob_weightings = -1/2 * np.sum(M @ self.ws.T, axis=1) - 1
            prob_weightings = prob_weightings/abs(np.sum(prob_weightings))
            self.prob = softmax(prob_weightings)
            self.probs = np.vstack([self.probs, self.prob.reshape(1, X.shape[0])])
            if np.isnan(self.probs).any():
                breakpoint()  
            print(percent_accuracy)
            percent_accuracy = np.sum(M @ np.mean(pna.ws, axis=0).T > 0)/M.shape[0]
            if((M @ np.mean(pna.ws, axis=0).T > 0).all()):
                break
                
        return np.mean(pna.ws, axis=0)

In [49]:
def get_dataset(dim):
    X = []
    y = []
    for i in range(dim):
        if len(X) == 0:
            vec = [0] * dim
            vec[0] = 1
            X.append(np.asarray(vec))
        else:
            vec = math.pow(-1, i) * abs(copy.deepcopy(X[i-1]))
            vec[i] = math.pow(-1, i+1)
            X.append(np.asarray(vec))
        y.append(math.pow(-1, i+1))
    return np.asarray(X), np.asarray(y)

In [50]:
n = 10
points = np.zeros((n,n))
labels = np.zeros(n)
for i in range(n):
    points[i,:i] = (-1)**(i+1)
    points[i,i] = (-1)**i
#     points[i,:] = points[i]/np.linalg.norm(points[i])
    labels[i] = (-1)**i
    
print(points)
print()
print(labels)
X = points
y = labels

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1. -1.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  1.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1. -1.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1. -1.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1.  1.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1. -1.]]

[ 1. -1.  1. -1.  1. -1.  1. -1.  1. -1.]


In [51]:
pna = Pnorm_accelerated(2)
# X, y= get_dataset(100)
M = X * y[:, np.newaxis]
final_w = pna.iterate(X, y)
accuracy = np.sum(M @ final_w > 0)/M.shape[0]
print(accuracy)




Acc: 0:   2%|▏         | 16/1000 [00:00<00:12, 80.84it/s]

0
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9


Acc: 0:   4%|▎         | 36/1000 [00:00<00:10, 90.02it/s]

0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9


Acc: 0:   6%|▌         | 56/1000 [00:00<00:10, 91.39it/s]

...

0.9
0.9
0.9


Acc: 0:  78%|███████▊  | 785/1000 [00:30<00:19, 11.22it/s]

0.9
0.9
0.9


Acc: 0:  78%|███████▊  | 785/1000 [00:30<00:08, 25.58it/s]


AttributeError: 'NoneType' object has no attribute 'reshape'

In [ ]:
num_examples = 100
# dim = 5
# X = np.random.rand(num_examples, dim)
# y = np.random.choice([-1, 1], (num_examples))
X, y= get_dataset(num_examples)
gentile = Gentile(3, .5, 1, 1)
final_weights = gentile.solve(X, y)
accuracy_metric = np.sum(y * (X @ final_weights) > 0)/X.shape[0]
print(y * X @ final_weights)
clf = Perceptron(tol=1e-3, random_state=0, fit_intercept=False)
clf.fit(X, y)
breakpoint()
accuracy_metric_scipy = y * np.squeeze(X @ clf.coef_.T)